In [27]:
import os
from google.cloud import storage
from matplotlib import pyplot as plt
import numpy as np
import sys

In [28]:
def get_item_data(item, N, seed, program, env):
    lst = list()
    if not os.path.isdir(f"./output/{env}"):
        os.system(f"mkdir ./output/{env}")
    if not os.path.isdir(f"./output/{env}/{program}"):
        os.system(f"mkdir ./output/{env}/{program}")
    for n in range(N):
        if not os.path.isdir(f"./outut/{env}/{program}/{seed + n}"):
            os.system(f"mkdir ./output/{env}/{program}/{seed + n}")
        bucket = storage.Client().bucket('kiran-research')
        blob = bucket.blob(f"latent-actions/outputs/{program}/seed{seed + n}/{env}/{item}.npy")
        blob.download_to_filename(f"./output/{env}/{program}/{seed + n}/{item}.npy")
        lst.append(np.load(f"./output/{env}/{program}/{seed + n}/{item}.npy"))
    print(f"from {program}: {[l.size for l in lst]}")
    arr = np.concatenate([l[np.newaxis, :] for l in lst], axis=0)
    return arr

In [36]:
N = 8
seed = 1970
item = "reward"
programs = ["bcq"]
colors = ["blue"]
env = 'halfcheetah-expert-v0'
plt.figure()
for i in range(len(programs)):
    rewards = get_item_data(item, N, seed, programs[i], env)
    mean = rewards.mean(-1)
    std = rewards.std(-1)
    eval_freq = 5000
    max_timesteps = 1000000
    color = sys.argv[i]
    program = sys.argv[i + 1]
    x = range(eval_freq, eval_freq + max_timesteps, eval_freq)
    plt.plot(x, mean, color=color, label=program)
    plt.fill_between(x, mean - std, mean + std, facecolor=color, alpha=0.2)
plt.legend()
plt.show()

from bcq: [88, 86, 87, 87, 89, 90, 59, 59]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 88 and the array at index 1 has size 86

<Figure size 432x288 with 0 Axes>